# Molecular dynamics from pretrained m3gnet model

In [5]:
import ase 
import warnings
import matgl
import os
import ML_library as MLL

from ase.io                      import read, write
from ase.io.vasp                 import write_vasp_xdatcar
from pymatgen.io.vasp.inputs     import Poscar
from pymatgen.core               import Structure, Lattice
from matgl.ext.ase               import Relaxer, MolecularDynamics
from pymatgen.io.ase             import AseAtomsAdaptor
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

for category in (UserWarning, DeprecationWarning):
    warnings.filterwarnings('ignore', category=category, module='tensorflow')

In [27]:
# Define basic parameters

#compound = 'LiMg'
#polymorf = 'Pm-3m'

temperature = 900  # In K
ensemble    = 'nvt'  # NVT canonical ensemble
timestep    = 1.5  # 1fs
steps       = 67000
loginterval = 10  # interval for record the log

# Define various paths

#path_to_folder     = f'Data/{compound}/{polymorf}'
path_to_folder     =  '/Users/cibran/Desktop/Li10GeS2P12/stc-0'
path_to_POSCAR     = f'{path_to_folder}/POSCAR'  # MP POSCAR (POSCAR_ini, DFT relaxed) is used
path_to_POSCAR_ini = f'{path_to_folder}/POSCAR_ini'

pot = matgl.load_model('M3GNet-MP-2021.2.8-PES')

# If POSCAR is not there, we use the POSCAR_ini (DFT relaxed)
#if not os.path.exists(path_to_POSCAR):
#    system(f'cp {path_to_folder}/POSCAR_ini {path_to_folder}')

trajectory  = f'{path_to_folder}/md.traj'  # save trajectory to mo.traj
logfile     = f'{path_to_folder}/md.log'  # log file for MD

# Write simulation parameters into INCAR
with open(f'{path_to_folder}/INCAR', 'w') as file:
    file.write(f'TEBEG = {temperature}\n')
    file.write(f'POTIM = {timestep}\n')
    file.write(f'NBLOCK = {loginterval}')
    file.truncate()

In [7]:
# Load the MP structure
# DFT relaxed structure, supposed to work better

if not os.path.exists(path_to_POSCAR):
    if os.path.exists(path_to_POSCAR_ini):
        # Copy POSCAR_ini as POSCAR
        os.system(f'cp {path_to_POSCAR_ini} {path_to_POSCAR}')

        # Relax POSCAR
        MLL.structural_relaxation(path_to_folder, 'M3GNet-MP-2021.2.8-PES', verbose=False)
        
        # Copy CONTCAR as POSCAR
        os.system(f'cp {path_to_folder}/CONTCAR {path_to_POSCAR}')
    else:
        sys.exit('Error: neither POSCAR nor POSCAR_ini files available.')

atoms = Structure.from_file(path_to_POSCAR)

In [29]:
ase_adaptor = AseAtomsAdaptor()

# Create ase atom object
md_atoms = ase_adaptor.get_atoms(atoms)

# Initialize the velocity according to Maxwell Boltzamnn distribution
MaxwellBoltzmannDistribution(md_atoms, temperature_K=temperature)

# Create the MD class
md = MolecularDynamics(
    atoms=md_atoms,
    potential=pot,
    temperature=temperature,
    ensemble=ensemble,
    timestep=timestep, 
    trajectory=trajectory,
    logfile=logfile,
    loginterval=loginterval
)

# Run
md.run(steps=steps)


KeyboardInterrupt



In [ ]:
# Read the .traj file
atoms = read(trajectory, format='traj')

# Access the atomic structure information
cell      = atoms.get_cell()
positions = atoms.get_positions()
symbols   = atoms.get_chemical_symbols()

In [6]:
# Convert results to XDATCAR format
ase_traj = ase.io.read(trajectory, index=':')
ase.io.vasp.write_vasp_xdatcar(f'{path_to_folder}/XDATCAR', ase_traj, label=None)